In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy as sp
import scipy.stats as stats
import nibabel as nibabel
import pandas as pd
import nibabel.freesurfer.mghformat as mgh
import scipy.io
import itertools 
import pickle

In [2]:
data_dir = '../../../data/'
local_data_dir = '../../../local_data/'

subjid = ['01', '02', '03', '04', '05', '06', '07', '08']
ROI_names = ['Unknown', 'Early', 'Midventral', 'Midlateral', 'Midparietal', 'Ventral', 'Lateral', 'Parietal']
n_repeats = 3

#threshold for voxels (based on split-half reliability)
thresh = 0.2

### Get right hemi data and process

In [3]:
#get ROI data
rh_streams = []
for sidx, sid in enumerate(subjid):
    mgh_file = mgh.load(data_dir+'nsddata/freesurfer/subj'+ sid +'/label/rh.streams.mgz')
    rh_streams.append(mgh_file.get_fdata()[:,0,0])

#get voxel level split-half reliability data
reliability = []
for sidx, sid in enumerate(subjid):
    
    sh_dir = local_data_dir + 'freesurfer/subj' + sid + '/rh_split_half.mat'
    sh = scipy.io.loadmat(sh_dir)
    
    reliability.append(sh['mean'])

#get organized z-scored betas
with open(local_data_dir + 'processed/rh_betas_by_repeat_by_ROI_zscore.data', 'rb') as filehandle:
    # read the data as binary data stream
    rh_betas_by_repeat_by_ROI = pickle.load(filehandle)

In [4]:
# Replace voxels with split-half reliability < thresh with NaNs and then trim those from data structure

sh_by_ROI = [[[] for j in range(len(ROI_names)-1)] for i in range(len(subjid))]
total_vox = np.zeros((len(subjid), len(ROI_names)-1))

#organize
for sidx, sid in enumerate(subjid):  
    for roi_idx in range(len(ROI_names)-1):       
        sh_by_ROI[sidx][roi_idx]=reliability[sidx][:,rh_streams[sidx] == roi_idx+1]
        total_vox[sidx,roi_idx] = len(sh_by_ROI[sidx][roi_idx][0])

#convert to nans
for sidx, sid in enumerate(subjid):  
    for roi_idx in range(len(ROI_names)-1): 
        for vox in range(len(sh_by_ROI[sidx][roi_idx][0])):
            if sh_by_ROI[sidx][roi_idx][0][vox] < thresh:
                rh_betas_by_repeat_by_ROI[sidx][roi_idx][0][:,vox]=np.nan
                rh_betas_by_repeat_by_ROI[sidx][roi_idx][1][:,vox]=np.nan
                rh_betas_by_repeat_by_ROI[sidx][roi_idx][2][:,vox]=np.nan    

thresh_vox = np.zeros((len(subjid), len(ROI_names)-1))                
#trim out nans
for sidx, sid in enumerate(subjid):   
    for roi_idx in range(len(ROI_names)-1): 
        for r in range(n_repeats):
            temp = rh_betas_by_repeat_by_ROI[sidx][roi_idx][r]
            trimmed = temp[:,~np.all(np.isnan(temp), axis=0)]

            rh_betas_by_repeat_by_ROI[sidx][roi_idx][r] = trimmed
        thresh_vox[sidx,roi_idx] = trimmed.shape[1]

#gather number of remaining voxels for each ROI and subj
vox = np.zeros((len(ROI_names)-1, len(subjid)))
for roi_idx in range(len(ROI_names)-1):
    vox[roi_idx,:] = [rh_betas_by_repeat_by_ROI[sidx][roi_idx][0].shape[1] for sidx, sid in enumerate(subjid)] 

In [5]:
# Create RSMS for all the ROIs, repeats and subjects

tril_flat_shape = int((rh_betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (rh_betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
rh_flat_rsm = np.zeros((len(subjid),len(ROI_names)-1, tril_flat_shape, n_repeats))
rsm = np.zeros((len(subjid),len(ROI_names)-1,n_repeats,rh_betas_by_repeat_by_ROI[0][0][0].shape[0],rh_betas_by_repeat_by_ROI[0][0][0].shape[0]))

for sidx, sid in enumerate(subjid):
    
    for roi_idx in range(len(ROI_names)-1):
        voxels = np.min(vox[roi_idx,:])
        
        for r in range(n_repeats):
            rsm[sidx,roi_idx,r,:,:] = np.corrcoef(rh_betas_by_repeat_by_ROI[sidx][roi_idx][r])
            lower = np.tril(rsm[sidx,roi_idx,r,:,:], -1).T.ravel() #only need lower triangle without diagonal
            rh_flat_rsm[sidx, roi_idx, :,r] = lower[lower != 0] #flatten into vectors

### Get left hemi data and process

In [ ]:
#get ROI data
lh_streams = []
for sidx, sid in enumerate(subjid):
    mgh_file = mgh.load(data_dir+'nsddata/freesurfer/subj'+ sid +'/label/lh.streams.mgz')
    lh_streams.append(mgh_file.get_fdata()[:,0,0])

#get voxel level split-half reliability data
reliability = []
for sidx, sid in enumerate(subjid):
    
    sh_dir = local_data_dir + 'freesurfer/subj' + sid + '/lh_split_half.mat'
    sh = scipy.io.loadmat(sh_dir)
    
    reliability.append(sh['mean'])

#get organized z-scored betas
with open(local_data_dir + 'processed/lh_betas_by_repeat_by_ROI_zscore.data', 'rb') as filehandle:
    # read the data as binary data stream
    lh_betas_by_repeat_by_ROI = pickle.load(filehandle)

In [ ]:
# Replace voxels with split-half reliability < thresh with NaNs and then trim those from data structure

sh_by_ROI = [[[] for j in range(len(ROI_names)-1)] for i in range(len(subjid))]
total_vox = np.zeros((len(subjid), len(ROI_names)-1))

#organize
for sidx, sid in enumerate(subjid):  
    for roi_idx in range(len(ROI_names)-1):       
        sh_by_ROI[sidx][roi_idx]=reliability[sidx][:,lh_streams[sidx] == roi_idx+1]
        total_vox[sidx,roi_idx] = len(sh_by_ROI[sidx][roi_idx][0])

#convert to nans
for sidx, sid in enumerate(subjid):  
    for roi_idx in range(len(ROI_names)-1): 
        for vox in range(len(sh_by_ROI[sidx][roi_idx][0])):
            if sh_by_ROI[sidx][roi_idx][0][vox] < thresh:
                lh_betas_by_repeat_by_ROI[sidx][roi_idx][0][:,vox]=np.nan
                lh_betas_by_repeat_by_ROI[sidx][roi_idx][1][:,vox]=np.nan
                lh_betas_by_repeat_by_ROI[sidx][roi_idx][2][:,vox]=np.nan    

                thresh_vox = np.zeros((len(subjid), len(ROI_names)-1))
                
#trim out nans
for sidx, sid in enumerate(subjid):   
    for roi_idx in range(len(ROI_names)-1): 
        for r in range(n_repeats):
            temp = lh_betas_by_repeat_by_ROI[sidx][roi_idx][r]
            trimmed = temp[:,~np.all(np.isnan(temp), axis=0)]

            lh_betas_by_repeat_by_ROI[sidx][roi_idx][r] = trimmed
        thresh_vox[sidx,roi_idx] = trimmed.shape[1]

#gather number of remaining voxels for each ROI and subj
vox = np.zeros((len(ROI_names)-1, len(subjid)))
for roi_idx in range(len(ROI_names)-1):
    vox[roi_idx,:] = [lh_betas_by_repeat_by_ROI[sidx][roi_idx][0].shape[1] for sidx, sid in enumerate(subjid)]        

In [ ]:
# Create RSMS for all the ROIs, repeats and subjects

tril_flat_shape = int((lh_betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (lh_betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
lh_flat_rsm = np.zeros((len(subjid),len(ROI_names)-1, tril_flat_shape, n_repeats))
rsm = np.zeros((len(subjid),len(ROI_names)-1,n_repeats,lh_betas_by_repeat_by_ROI[0][0][0].shape[0],lh_betas_by_repeat_by_ROI[0][0][0].shape[0]))

for sidx, sid in enumerate(subjid):
    
    for roi_idx in range(len(ROI_names)-1):
        voxels = np.min(vox[roi_idx,:])
        
        for r in range(n_repeats):
            rsm[sidx,roi_idx,r,:,:] = np.corrcoef(lh_betas_by_repeat_by_ROI[sidx][roi_idx][r])
            lower = np.tril(rsm[sidx,roi_idx,r,:,:], -1).T.ravel() #only need lower triangle without diagonal
            lh_flat_rsm[sidx, roi_idx, :,r] = lower[lower != 0] #flatten into vectors

## Compare hemispheres

In [ ]:
r1_trial_order = [0, 0, 0, 1, 1, 1, 2, 2, 2]
r2_trial_order = [0, 1, 2, 0, 1, 2, 0, 1, 2]

#don't use same trials when same subject
ss_r1_trial_order = [0, 0, 1, 1, 2, 2]
ss_r2_trial_order = [1, 2, 0, 2, 0, 1]

In [ ]:
total_combs = len(subjid) * (len(ROI_names)-1)
mega_matrix = np.zeros((total_combs,total_combs))
total_rsms_sh_rh = np.zeros((len(subjid),len(ROI_names)-1))
total_rsms_sh_lh = np.zeros((len(subjid),len(ROI_names)-1))
total_rsms_across_hemis = np.zeros((len(subjid),len(ROI_names)-1))


for i in range(total_combs): #rows - i.e. model candidate
    roi_idx1 = i%7 #roi
    sidx1 = i//7 #subject
    
    #right hemisphere split half
    split_half = np.zeros((3))
    split_half = [stats.pearsonr(rh_flat_rsm[sidx1,roi_idx1,:,0],rh_flat_rsm[sidx1,roi_idx1,:,1])[0],
                  stats.pearsonr(rh_flat_rsm[sidx1,roi_idx1,:,0],rh_flat_rsm[sidx1,roi_idx1,:,2])[0],
                  stats.pearsonr(rh_flat_rsm[sidx1,roi_idx1,:,1],rh_flat_rsm[sidx1,roi_idx1,:,2])[0]]
    total_rsms_sh_rh[sidx1,roi_idx1]= np.mean(split_half)
    
    #left hemisphere split half
    split_half = np.zeros((3))
    split_half = [stats.pearsonr(lh_flat_rsm[sidx1,roi_idx1,:,0],lh_flat_rsm[sidx1,roi_idx1,:,1])[0],
                  stats.pearsonr(lh_flat_rsm[sidx1,roi_idx1,:,0],lh_flat_rsm[sidx1,roi_idx1,:,2])[0],
                  stats.pearsonr(lh_flat_rsm[sidx1,roi_idx1,:,1],lh_flat_rsm[sidx1,roi_idx1,:,2])[0]]
    total_rsms_sh_lh[sidx1,roi_idx1]= np.mean(split_half)
    
    for j in range(total_combs): #columns - i.e. target data
        roi_idx2 = j%7 #roi
        sidx2 = j//7 #subject
        
        if (sidx1 == sidx2) and (roi_idx1 == roi_idx2): #within subject and roi only
            rsm_corr = np.zeros((6))
            for r in range(6):
                rsm_corr[r] = np.absolute(stats.pearsonr(rh_flat_rsm[sidx1,roi_idx1,:,ss_r1_trial_order[r]],
                                                         lh_flat_rsm[sidx2,roi_idx2,:,ss_r2_trial_order[r]])[0])
            total_rsms_across_hemis[sidx1, roi_idx1] = np.mean(rsm_corr)
        
        

In [ ]:
total_rsms_across_hemis[:,4]

In [ ]:
total_rsms_sh_rh[:,4]

In [ ]:
total_rsms_sh_lh[:,4]

In [ ]:
np.mean(total_rsms_across_hemis, axis=0)

In [ ]:
np.mean(total_rsms_sh_rh, axis=0)

In [ ]:
np.mean(total_rsms_sh_lh, axis=0)

In [ ]:

labels = ['Early', 'Midventral', 'Midlateral', 'Midparietal', 'Ventral', 'Lateral', 'Parietal']
total_means = np.mean(total_rsms_across_hemis, axis=0)
rh_means = np.mean(total_rsms_sh_rh, axis=0)
lh_means = np.mean(total_rsms_sh_lh, axis=0)

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(12,5))

rects1 = ax.bar(x - 1.01*width, lh_means, width, label='Left', color='navy')
rects2 = ax.bar(x, rh_means, width, label='Right', color='blue')
rects3 = ax.bar(x + 1.01*width, total_means, width, label='Across', color='orchid')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Pearsons r',fontsize=20)
ax.set_title('RSM correlation within and across hemispheres ('+str(thresh*100)+'% threshold)',fontsize=20)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height,2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.ylim(0,.6)
plt.show()

fig.savefig('../../../results/figures/hemi_comparison_voxThresh' + str(int(thresh*100)) + '.png')
